In [ ]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
cpp_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/moberg_extracted_data/cpp_df.csv'
ptio2_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/moberg_extracted_data/ptio2_df.csv'
filtered_ptio2_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/moberg_extracted_data/ptio2_df_filtered.csv'
temperature_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/moberg_extracted_data/temperature_df.csv'
lpr_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/moberg_extracted_data/lpr_df.csv'
hr_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/moberg_extracted_data/hr_df.csv'
etco2_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/moberg_extracted_data/etco2_df.csv'
ci_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/moberg_extracted_data/ci_df.csv'
prx_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/moberg_extracted_data/prx_df.csv'
drug_administration_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/drug_administrations.xlsx'
registry_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/moberg_registry_kssg_post_hoc_modified.xlsx'
paco2_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/pdms_data/joined_aBGA.csv'
mainstream_etco2_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/neurocrit_fever/data/PDMS_data/joined_etCO2.csv'
icp_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/neurocrit_fever/data/moberg_extracted_data/icp_df.csv'

In [ ]:
exclude_short_infusions = True
use_filtered_ptio2 = True

In [ ]:
cpp_df = pd.read_csv(cpp_path)
if use_filtered_ptio2:
    ptio2_df = pd.read_csv(filtered_ptio2_path)
else:
    ptio2_df = pd.read_csv(ptio2_path)
temperature_df = pd.read_csv(temperature_path)
lpr_df = pd.read_csv(lpr_path)
hr_df = pd.read_csv(hr_path)
prx_df = pd.read_csv(prx_path)
icp_df = pd.read_csv(icp_path)
drug_administration_df = pd.read_excel(drug_administration_path)

In [ ]:
registry_df = pd.read_excel(registry_path)
paco2_df = pd.read_csv(paco2_path, sep=';')
mainstream_etco2_df = pd.read_csv(mainstream_etco2_path, sep=';')

In [ ]:
paco2_df = pd.merge(paco2_df, registry_df[['manual_mrn', 'Pat. Nr.']], left_on='FallNr', right_on='manual_mrn',
                    how='left')
paco2_df.drop(columns=['manual_mrn'], inplace=True)
paco2_df.rename(columns={'Pat. Nr.': 'pat_nr'}, inplace=True)
paco2_df.rename(columns={'Zeitpunkt_aBGA': 'datetime'}, inplace=True)
paco2_df['pCO2_mmHg'] = paco2_df['pCO2'] * 7.50062

mainstream_etco2_df = pd.merge(mainstream_etco2_df, registry_df[['manual_mrn', 'Pat. Nr.']], left_on='FallNr', right_on='manual_mrn', how='left')
mainstream_etco2_df.drop(columns=['manual_mrn'], inplace=True)
mainstream_etco2_df.rename(columns={'Pat. Nr.':'pat_nr'}, inplace=True)
mainstream_etco2_df.rename(columns={'Zeitpunkt_etCO2':'datetime'}, inplace=True)

In [ ]:
drug_administration_df = drug_administration_df[drug_administration_df.monitored]

n_patients_before = drug_administration_df['pat_nr'].nunique()
# print patients with exclusion criterium
print(f'Excluding {drug_administration_df[~pd.isna(drug_administration_df["further_exclusion_criterium"])].shape[0]} infusions with {drug_administration_df[~pd.isna(drug_administration_df["further_exclusion_criterium"])]["further_exclusion_criterium"].nunique()} different further exclusion criteria')
# exclude if further_exclusion_criterium is not Nan
drug_administration_df = drug_administration_df[pd.isna(drug_administration_df['further_exclusion_criterium'])]
# print number of patients excluded
print(f'Excluding {n_patients_before - drug_administration_df["pat_nr"].nunique()} patients with further exclusion criterium')

if exclude_short_infusions:
    n_patients_before = drug_administration_df['pat_nr'].nunique()
    drug_administration_df['infusion_duration'] = (pd.to_datetime(drug_administration_df['drug_end']) - pd.to_datetime(drug_administration_df['drug_start'])).dt.total_seconds() / 3600
    print(f'Excluding {drug_administration_df[drug_administration_df["infusion_duration"] <= 1].shape[0]} infusions with duration <= 1h')
    drug_administration_df = drug_administration_df[drug_administration_df['infusion_duration'] > 1]
    print(f'Excluding {n_patients_before - drug_administration_df["pat_nr"].nunique()} patients with infusions with duration <= 1h')

In [ ]:
for var_df in [ptio2_df, cpp_df, temperature_df, lpr_df, hr_df, paco2_df, prx_df, mainstream_etco2_df, icp_df]:
    var_df['datetime'] = pd.to_datetime(var_df['datetime'])

In [ ]:
# for every drug administration extract data from -xh to +xh around start
time_window_post = 12
time_window_pre = 12

associated_ptio2_df = pd.DataFrame()
associated_cpp_df = pd.DataFrame()
associated_temperature_df = pd.DataFrame()
associated_hr_df = pd.DataFrame()
associated_lpr_df = pd.DataFrame()
associated_ci_df = pd.DataFrame()
associated_prx_df = pd.DataFrame()
associated_etco2_df = pd.DataFrame()
associated_paco2_df = pd.DataFrame()
associated_mainstream_etco2_df = pd.DataFrame()
associated_icp_df = pd.DataFrame()

for index, row in drug_administration_df.iterrows():
    lower_bound = row['drug_start'] - pd.to_timedelta(time_window_pre, unit='h')
    upper_bound = row['drug_start'] + pd.to_timedelta(time_window_post, unit='h')
    instance_associated_ptio2_df = ptio2_df[(ptio2_df['pat_nr'] == row['pat_nr'])
                                            & (ptio2_df['datetime'] >= lower_bound) 
                                            & (ptio2_df['datetime'] <= upper_bound)]
    instance_associated_ptio2_df['drug_start'] = row['drug_start']
    instance_associated_ptio2_df['relative_datetime'] = (instance_associated_ptio2_df['datetime'] - row['drug_start']).dt.total_seconds() / 3600
    associated_ptio2_df = pd.concat([associated_ptio2_df, instance_associated_ptio2_df])

    instance_associated_cpp_df = cpp_df[(cpp_df['pat_nr'] == row['pat_nr'])
                                        & (cpp_df['datetime'] >= lower_bound) 
                                        & (cpp_df['datetime'] <= upper_bound)]  
    instance_associated_cpp_df['drug_start'] = row['drug_start']
    instance_associated_cpp_df['relative_datetime'] = (instance_associated_cpp_df['datetime'] - row['drug_start']).dt.total_seconds() / 3600
    associated_cpp_df = pd.concat([associated_cpp_df, instance_associated_cpp_df])

    instance_associated_temperature_df = temperature_df[(temperature_df['pat_nr'] == row['pat_nr'])
                                        & (temperature_df['datetime'] >= lower_bound)
                                        & (temperature_df['datetime'] <= upper_bound)]
    instance_associated_temperature_df['drug_start'] = row['drug_start']
    instance_associated_temperature_df['relative_datetime'] = (instance_associated_temperature_df['datetime'] - row['drug_start']).dt.total_seconds() / 3600
    associated_temperature_df = pd.concat([associated_temperature_df, instance_associated_temperature_df])

    instance_associated_hr_df = hr_df[(hr_df['pat_nr'] == row['pat_nr'])
                                        & (hr_df['datetime'] >= lower_bound)   
                                        & (hr_df['datetime'] <= upper_bound)]
    instance_associated_hr_df['drug_start'] = row['drug_start']
    instance_associated_hr_df['relative_datetime'] = (instance_associated_hr_df['datetime'] - row['drug_start']).dt.total_seconds() / 3600
    associated_hr_df = pd.concat([associated_hr_df, instance_associated_hr_df])

    instance_associated_lpr_df = lpr_df[(lpr_df['pat_nr'] == row['pat_nr'])
                                        & (lpr_df['datetime'] >= lower_bound)
                                        & (lpr_df['datetime'] <= upper_bound)]
    instance_associated_lpr_df['drug_start'] = row['drug_start']
    instance_associated_lpr_df['relative_datetime'] = (instance_associated_lpr_df['datetime'] - row['drug_start']).dt.total_seconds() / 3600
    associated_lpr_df = pd.concat([associated_lpr_df, instance_associated_lpr_df])
    
    instance_associated_paco2_df = paco2_df[(paco2_df['pat_nr'] == row['pat_nr'])
                                        & (paco2_df['datetime'] >= lower_bound)
                                        & (paco2_df['datetime'] <= upper_bound)]
    instance_associated_paco2_df['drug_start'] = row['drug_start']
    instance_associated_paco2_df['relative_datetime'] = (instance_associated_paco2_df['datetime'] - row['drug_start']).dt.total_seconds() / 3600
    associated_paco2_df = pd.concat([associated_paco2_df, instance_associated_paco2_df])
    
    instance_associated_prx_df = prx_df[(prx_df['pat_nr'] == row['pat_nr'])
                                    & (prx_df['datetime'] >= lower_bound)
                                    & (prx_df['datetime'] <= upper_bound)]
    instance_associated_prx_df['drug_start'] = row['drug_start']
    instance_associated_prx_df['relative_datetime'] = (instance_associated_prx_df['datetime'] - row['drug_start']).dt.total_seconds() / 3600
    associated_prx_df = pd.concat([associated_prx_df, instance_associated_prx_df])
    
    instance_associated_mainstream_etco2_df = mainstream_etco2_df[(mainstream_etco2_df['pat_nr'] == row['pat_nr'])
                                        & (mainstream_etco2_df['datetime'] >= lower_bound)
                                        & (mainstream_etco2_df['datetime'] <= upper_bound)]
    instance_associated_mainstream_etco2_df['drug_start'] = row['drug_start']
    instance_associated_mainstream_etco2_df['relative_datetime'] = (instance_associated_mainstream_etco2_df['datetime'] - row['drug_start']).dt.total_seconds() / 3600
    associated_mainstream_etco2_df = pd.concat([associated_mainstream_etco2_df, instance_associated_mainstream_etco2_df])

    instance_associated_icp_df = icp_df[(icp_df['pat_nr'] == row['pat_nr'])
                                        & (icp_df['datetime'] >= lower_bound)
                                        & (icp_df['datetime'] <= upper_bound)]
    instance_associated_icp_df['drug_start'] = row['drug_start']
    instance_associated_icp_df['relative_datetime'] = (instance_associated_icp_df['datetime'] - row['drug_start']).dt.total_seconds() / 3600
    associated_icp_df = pd.concat([associated_icp_df, instance_associated_icp_df])

    

In [ ]:
for var_df in [associated_ptio2_df, associated_cpp_df, associated_temperature_df, associated_hr_df, associated_icp_df]:
    # remove duplicates
    # var_df.drop_duplicates(subset=['pat_nr', 'relative_datetime'], inplace=True)
    # round to 2 decimals of an hour
    var_df['relative_datetime_cat'] = var_df['relative_datetime'].round(2)

In [ ]:
grouped_ptio2 = associated_ptio2_df.groupby(['pat_nr', 'drug_start', 'relative_datetime_cat']).agg({'ptio2': 'median'}).reset_index()
grouped_cpp = associated_cpp_df.groupby(['pat_nr', 'drug_start', 'relative_datetime_cat']).agg({'cpp': 'median'}).reset_index()
grouped_temperature = associated_temperature_df.groupby(['pat_nr', 'drug_start', 'relative_datetime_cat']).agg({'temperature': 'median'}).reset_index()
grouped_hr = associated_hr_df.groupby(['pat_nr', 'drug_start', 'relative_datetime_cat']).agg({'hr': 'median'}).reset_index()
grouped_icp = associated_icp_df.groupby(['pat_nr', 'drug_start', 'relative_datetime_cat']).agg({'icp': 'median'}).reset_index()
# grouped_lpr = associated_lpr_df.groupby(['pat_nr', 'drug_start', 'relative_datetime_cat']).agg({'lpr': 'median'}).reset_index()
# grouped_paco2 = associated_paco2_df.groupby(['pat_nr', 'drug_start', 'relative_datetime_cat']).agg({'pCO2_mmHg': 'median'}).reset_index()

In [ ]:
# merge
concatenated_df = grouped_ptio2.merge(grouped_cpp, on=['pat_nr', 'drug_start', 'relative_datetime_cat'], how='outer')
concatenated_df = concatenated_df.merge(grouped_temperature, on=['pat_nr', 'drug_start', 'relative_datetime_cat'], how='outer')
concatenated_df = concatenated_df.merge(grouped_hr, on=['pat_nr', 'drug_start', 'relative_datetime_cat'], how='outer')
# concatenated_df = concatenated_df.merge(grouped_icp, on=['pat_nr', 'drug_start', 'relative_datetime_cat'], how='outer')
# concatenated_df = concatenated_df.merge(grouped_lpr, on=['pat_nr', 'drug_start', 'relative_datetime_cat'], how='outer')
# concatenated_df = concatenated_df.merge(grouped_paco2, on=['pat_nr', 'drug_start', 'relative_datetime_cat'], how='outer')

In [ ]:
concatenated_df['pre_post'] = 'pre'
concatenated_df.loc[concatenated_df['relative_datetime_cat'] >= 0, 'pre_post'] = 'post'

## Multivariate model
create full mixed effects model with all variables and interactions

In [ ]:
temp_df = concatenated_df
# drop rows with nan values
temp_df = temp_df.dropna()

In [ ]:
mixed_model = smf.mixedlm("ptio2 ~ pre_post * cpp * temperature * hr", temp_df, groups=temp_df['pat_nr'])
mixed_model_fit = mixed_model.fit()

print(mixed_model.score(mixed_model_fit.params_object))
print(mixed_model_fit.summary())

In [ ]:
print(mixed_model_fit.pvalues)

# Single variable models 

In [ ]:
# ptio2
temp_ptio2_df = concatenated_df.drop(columns=['cpp', 'temperature', 'hr'])
# drop rows with nan values
temp_ptio2_df = temp_df.dropna()

mixed_model_ptio2 = smf.mixedlm("ptio2 ~ pre_post", temp_ptio2_df, groups=temp_ptio2_df['pat_nr'])

In [ ]:
mixed_model_ptio2_fit = mixed_model_ptio2.fit()
mixed_model_ptio2_fit.summary()

In [ ]:
mixed_model_ptio2_fit.pvalues

In [ ]:
# pre / post median and IQR
print(f'Pre: {concatenated_df[concatenated_df["pre_post"] == "pre"]["ptio2"].median()} ({concatenated_df[concatenated_df["pre_post"] == "pre"]["ptio2"].quantile(0.25)}, {concatenated_df[concatenated_df["pre_post"] == "pre"]["ptio2"].quantile(0.75)})',
f'Post: {concatenated_df[concatenated_df["pre_post"] == "post"]["ptio2"].median()} ({concatenated_df[concatenated_df["pre_post"] == "post"]["ptio2"].quantile(0.25)}, {concatenated_df[concatenated_df["pre_post"] == "post"]["ptio2"].quantile(0.75)})')

In [ ]:
# percentual median decrease
print(f'{(1 - concatenated_df[concatenated_df["pre_post"] == "post"]["ptio2"].median() / concatenated_df[concatenated_df["pre_post"] == "pre"]["ptio2"].median()) * 100:.2f}%')

In [ ]:
# percentual mean decrease
print(f'{(1 - concatenated_df[concatenated_df["pre_post"] == "post"]["ptio2"].mean() / concatenated_df[concatenated_df["pre_post"] == "pre"]["ptio2"].mean()) * 100:.2f}%')

In [ ]:
# cpp
temp_cpp_df = concatenated_df.drop(columns=['ptio2', 'temperature', 'hr'])
# drop rows with nan values
temp_cpp_df = temp_df.dropna()

In [ ]:
mixed_model_cpp = smf.mixedlm("cpp ~ pre_post", temp_cpp_df, groups=temp_cpp_df['pat_nr'])

In [ ]:
mixed_model_cpp_fit = mixed_model_cpp.fit()
mixed_model_cpp_fit.summary()

In [ ]:
mixed_model_cpp_fit.pvalues

In [ ]:
# pre / post median and IQR
print(f'Pre: {concatenated_df[concatenated_df["pre_post"] == "pre"]["cpp"].median()} ({concatenated_df[concatenated_df["pre_post"] == "pre"]["cpp"].quantile(0.25)}, {concatenated_df[concatenated_df["pre_post"] == "pre"]["cpp"].quantile(0.75)})',
f'Post: {concatenated_df[concatenated_df["pre_post"] == "post"]["cpp"].median()} ({concatenated_df[concatenated_df["pre_post"] == "post"]["cpp"].quantile(0.25)}, {concatenated_df[concatenated_df["pre_post"] == "post"]["cpp"].quantile(0.75)})')

In [ ]:
# pre/post mean and IQR
print(f'Pre: {concatenated_df[concatenated_df["pre_post"] == "pre"]["cpp"].mean()} ({concatenated_df[concatenated_df["pre_post"] == "pre"]["cpp"].quantile(0.25)}, {concatenated_df[concatenated_df["pre_post"] == "pre"]["cpp"].quantile(0.75)})',
f'Post: {concatenated_df[concatenated_df["pre_post"] == "post"]["cpp"].mean()} ({concatenated_df[concatenated_df["pre_post"] == "post"]["cpp"].quantile(0.25)}, {concatenated_df[concatenated_df["pre_post"] == "post"]["cpp"].quantile(0.75)})')

In [ ]:
# percentual mean decrease
print(f'{(1 - concatenated_df[concatenated_df["pre_post"] == "post"]["cpp"].mean() / concatenated_df[concatenated_df["pre_post"] == "pre"]["cpp"].mean()) * 100:.2f}%')

In [ ]:
# temperature
temp_temperature_df = concatenated_df.drop(columns=['cpp', 'ptio2', 'hr'])
# drop rows with nan values
temp_temperature_df = temp_df.dropna()

In [ ]:
mixed_model_temperature = smf.mixedlm("temperature ~ pre_post", temp_temperature_df, groups=temp_temperature_df['pat_nr'])

In [ ]:
mixed_model_temperature_fit = mixed_model_temperature.fit()
mixed_model_temperature_fit.summary()

In [ ]:
mixed_model_temperature_fit.pvalues

In [ ]:
# pre / post median and IQR
print(f'Pre: {concatenated_df[concatenated_df["pre_post"] == "pre"]["temperature"].median()} ({concatenated_df[concatenated_df["pre_post"] == "pre"]["temperature"].quantile(0.25)}, {concatenated_df[concatenated_df["pre_post"] == "pre"]["temperature"].quantile(0.75)})',
f'Post: {concatenated_df[concatenated_df["pre_post"] == "post"]["temperature"].median()} ({concatenated_df[concatenated_df["pre_post"] == "post"]["temperature"].quantile(0.25)}, {concatenated_df[concatenated_df["pre_post"] == "post"]["temperature"].quantile(0.75)})')

In [ ]:
# hr
temp_hr_df = concatenated_df.drop(columns=['cpp', 'temperature', 'ptio2'])
# drop rows with nan values
temp_hr_df = temp_df.dropna()

In [ ]:
mixed_model_hr = smf.mixedlm("hr ~ pre_post", temp_hr_df, groups=temp_hr_df['pat_nr'])

In [ ]:
mixed_model_hr_fit = mixed_model_hr.fit()
mixed_model_hr_fit.summary()

In [ ]:
mixed_model_hr_fit.pvalues

In [ ]:
# pre / post median and IQR
print(f'Pre: {concatenated_df[concatenated_df["pre_post"] == "pre"]["hr"].median()} ({concatenated_df[concatenated_df["pre_post"] == "pre"]["hr"].quantile(0.25)}, {concatenated_df[concatenated_df["pre_post"] == "pre"]["hr"].quantile(0.75)})',
f'Post: {concatenated_df[concatenated_df["pre_post"] == "post"]["hr"].median()} ({concatenated_df[concatenated_df["pre_post"] == "post"]["hr"].quantile(0.25)}, {concatenated_df[concatenated_df["pre_post"] == "post"]["hr"].quantile(0.75)})')

paco2

In [ ]:
associated_paco2_df['pre_post'] = 'pre'
associated_paco2_df.loc[associated_paco2_df['relative_datetime'] >= 0, 'pre_post'] = 'post'

In [ ]:
associated_paco2_df.pat_nr.nunique(), associated_paco2_df.drug_start.nunique()

In [ ]:
mixed_model_paco2 = smf.mixedlm("pCO2_mmHg ~ pre_post", associated_paco2_df, groups=associated_paco2_df['pat_nr'])

In [ ]:
mixed_model_paco2_fit = mixed_model_paco2.fit()
mixed_model_paco2_fit.summary()

In [ ]:
mixed_model_paco2_fit.pvalues

In [ ]:
# pre / post median and IQR
print(f'Pre: {associated_paco2_df[associated_paco2_df["pre_post"] == "pre"]["pCO2_mmHg"].median()} ({associated_paco2_df[associated_paco2_df["pre_post"] == "pre"]["pCO2_mmHg"].quantile(0.25)}, {associated_paco2_df[associated_paco2_df["pre_post"] == "pre"]["pCO2_mmHg"].quantile(0.75)})',
f'Post: {associated_paco2_df[associated_paco2_df["pre_post"] == "post"]["pCO2_mmHg"].median()} ({associated_paco2_df[associated_paco2_df["pre_post"] == "post"]["pCO2_mmHg"].quantile(0.25)}, {associated_paco2_df[associated_paco2_df["pre_post"] == "post"]["pCO2_mmHg"].quantile(0.75)})')

Mainstream etCO2

In [ ]:
associated_mainstream_etco2_df['pre_post'] = 'pre'
associated_mainstream_etco2_df.loc[associated_mainstream_etco2_df['relative_datetime'] >= 0, 'pre_post'] = 'post'

In [ ]:
associated_mainstream_etco2_df.pat_nr.nunique(), associated_mainstream_etco2_df.drug_start.nunique()

In [ ]:
mixed_model_mainstream_etco2 = smf.mixedlm("etCO2 ~ pre_post", associated_mainstream_etco2_df, groups=associated_mainstream_etco2_df['pat_nr'])

In [ ]:
mixed_model_mainstream_etco2_fit = mixed_model_mainstream_etco2.fit()
mixed_model_mainstream_etco2_fit.summary()

In [ ]:
mixed_model_mainstream_etco2_fit.pvalues

In [ ]:
# pre / post median and IQR
print(f'Pre: {associated_mainstream_etco2_df[associated_mainstream_etco2_df["pre_post"] == "pre"]["etCO2"].median()} ({associated_mainstream_etco2_df[associated_mainstream_etco2_df["pre_post"] == "pre"]["etCO2"].quantile(0.25)}, {associated_mainstream_etco2_df[associated_mainstream_etco2_df["pre_post"] == "pre"]["etCO2"].quantile(0.75)})',
f'Post: {associated_mainstream_etco2_df[associated_mainstream_etco2_df["pre_post"] == "post"]["etCO2"].median()} ({associated_mainstream_etco2_df[associated_mainstream_etco2_df["pre_post"] == "post"]["etCO2"].quantile(0.25)}, {associated_mainstream_etco2_df[associated_mainstream_etco2_df["pre_post"] == "post"]["etCO2"].quantile(0.75)})')

LPR subset

In [ ]:
associated_lpr_df['pre_post'] = 'pre'
associated_lpr_df.loc[associated_lpr_df['relative_datetime'] >= 0, 'pre_post'] = 'post'
associated_lpr_df.dropna(subset=['lpr'], inplace=True)

In [ ]:
associated_lpr_df.pat_nr.nunique(), associated_lpr_df.drug_start.nunique()

In [ ]:
mixed_model_lpr = smf.mixedlm("lpr ~ pre_post", associated_lpr_df, groups=associated_lpr_df['pat_nr'])

In [ ]:
mixed_model_lpr_fit = mixed_model_lpr.fit()
mixed_model_lpr_fit.summary()

In [ ]:
mixed_model_lpr_fit.pvalues

In [ ]:
f'{mixed_model_lpr_fit.pvalues["pre_post[T.pre]"]:.4f}'

In [ ]:
# pre / post median and IQR
print(f'Pre: {concatenated_df[concatenated_df["pre_post"] == "pre"]["lpr"].median()} ({concatenated_df[concatenated_df["pre_post"] == "pre"]["lpr"].quantile(0.25)}, {concatenated_df[concatenated_df["pre_post"] == "pre"]["lpr"].quantile(0.75)})',
f'Post: {concatenated_df[concatenated_df["pre_post"] == "post"]["lpr"].median()} ({concatenated_df[concatenated_df["pre_post"] == "post"]["lpr"].quantile(0.25)}, {concatenated_df[concatenated_df["pre_post"] == "post"]["lpr"].quantile(0.75)})')

PRX

In [ ]:
associated_prx_df['pre_post'] = 'pre'
associated_prx_df.loc[associated_prx_df['relative_datetime'] >= 0, 'pre_post'] = 'post'

In [ ]:
associated_prx_df.pat_nr.nunique(), associated_prx_df.drug_start.nunique() 

In [ ]:
associated_prx_df

In [ ]:
mixed_model_lpr = smf.mixedlm("prx ~ pre_post", associated_prx_df, groups=associated_prx_df['pat_nr'])

In [ ]:
mixed_model_lpr_fit = mixed_model_lpr.fit()
mixed_model_lpr_fit.summary()

In [ ]:
mixed_model_lpr_fit.pvalues

In [ ]:
# pre / post median and IQR
print(f'Pre: {associated_prx_df[associated_prx_df["pre_post"] == "pre"]["prx"].median():.2f} ({associated_prx_df[associated_prx_df["pre_post"] == "pre"]["prx"].quantile(0.25):.2f}, {associated_prx_df[associated_prx_df["pre_post"] == "pre"]["prx"].quantile(0.75):.2f})',
f'Post: {associated_prx_df[associated_prx_df["pre_post"] == "post"]["prx"].median():.2f} ({associated_prx_df[associated_prx_df["pre_post"] == "post"]["prx"].quantile(0.25):.2f}, {associated_prx_df[associated_prx_df["pre_post"] == "post"]["prx"].quantile(0.75):.2f})')